# Victorian Biodiversity Atlas

These two datasets contain surveys of biota in the Victorian area collected by DEECA (see (VBA Fauna metadata)[https://metashare.maps.vic.gov.au/geonetwork/srv/api/records/84b924b3-e115-58d9-b707-2920143051f7/formatters/sdm-html?root=html&output=html] and (VBA Flora metadata)[https://metashare.maps.vic.gov.au/geonetwork/srv/api/records/4803b0e0-e312-5371-8e6a-88248e42e17d/formatters/sdm-html?root=html&output=html] for more details).
                                                                                      
The original datasets were modified by adding a column (`affected`) that indicates thae number of times a certain biota was within the burnt area during the Black Summer of 2019-2020 (there are cases of multiple fires at the same location).
                                                                                      

In [31]:
library(pivottabler)
library(IRdisplay)

flo.df <- read.csv('./data/vba/flora.csv')
fau.df <- read.csv('./data/vba/fauna.csv')

recoder <- function (d) {
  d$vicadvdesc <- ifelse(d$vicadvdesc == '', '<empty>', d$vicadvdesc)
  d$affected_flag <- ifelse(d$affected > 0, 'Affected', 'Not affected')
  d$affected_sum <- ifelse(d$affected > 0, 1, 0)
  d
}

flo.df <- recoder(flo.df)
fau.df <- recoder(fau.df)


In [34]:
unique(flo.df$vicadvdesc)
unique(fau.df$vicadvdesc)

[1] "<empty>"            "Rare"               "Vulnerable"        
[4] "All ssp threatened" "Poorly known"       "Endangered"        
[7] "Presumed extinct"

[1] "Vulnerable"            "<empty>"               "Near threatened"      
[4] "Endangered"            "Critically endangered" "Data deficient"       
[7] "Extinct in the Wild"   "Regionally extinct"    "Presumed extinct"

In [37]:

#eflo.df <- subset(flo.df, vicadvdesc %in% c('Rare', 'Endangered', 'Presumed extinct', 'Vulnerable', 'All ssp threatened'))
pivot <- function (d) {
  pt <- PivotTable$new()
  pt$addData(d)
  #pt$addColumnDataGroups("affected_flag", totalCaption="Total")
  pt$addRowDataGroups("vicadvdesc", totalCaption="Total")
  #pt$addRowDataGroups("comm_name", totalCaption="Name")
  #pt$defineCalculation(calculationName="Affected", caption="Affected count", 
  #                     summariseExpression="sum(affected)")
  pt$defineCalculation(calculationName="CountAffected", caption="Affected Biota count", 
                       summariseExpression="sum(affected_sum)")
  pt$defineCalculation(calculationName="Count", caption="Biota count", 
                       summariseExpression="n()")
  pt$defineCalculation(calculationName="PctAffected", caption="% Affected Biota", 
                       type="calculation", basedOn=c("CountAffected", "Count"), 
                       format="%.1f %%",
                       calculationExpression="values$CountAffected/values$Count*100")
  pt$evaluatePivot()
  pt$renderPivot()
}

display_markdown('## Flora')
pivot(flo.df)

display_markdown('## Fauna')
pivot(fau.df)

## Flora

HTML widgets cannot be represented in plain text (need html)

## Fauna

HTML widgets cannot be represented in plain text (need html)